In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using OrdinaryDiffEq
using ForwardDiff
using Plots

In [ ]:
#Parameters
g = 9.81
m = 1.0 #link mass
ℓ = 1.0 #link length
J = 0.5*m*ℓ*ℓ #link inertia

In [ ]:
function kinematics(q)
    
    r = [0.5*ℓ*sin(q[1]); -0.5*ℓ*cos(q[1]); q[1]; 
         ℓ*sin(q[1]) + 0.5*ℓ*sin(q[1]+q[2]);
         -ℓ*cos(q[1]) - 0.5*ℓ*cos(q[1]+q[2]);
         q[1]+q[2]]
end

In [ ]:
function potential(q)
    
    r = kinematics(q)
    y1 = r[2]
    y2 = r[5]
    
    U = m*g*y1 + m*g*y2
end

In [ ]:
function kinetic(q,q̇)
    
    K = ForwardDiff.jacobian(kinematics,q)
    ṙ = K*q̇
    
    T = 0.5*ṙ'*Diagonal([m; m; J; m; m; J])*ṙ
end

In [ ]:
function Lagrangian(q,q̇)
    return kinetic(q,q̇) - potential(q)
end

In [ ]:
function D1L(q,q̇)
    return ForwardDiff.gradient(dq->Lagrangian(dq,q̇),q)
end

function D2L(q,q̇)
    return ForwardDiff.gradient(dq̇->Lagrangian(q,dq̇),q̇)
end

function D2D2L(q,q̇)
    return ForwardDiff.jacobian(dq̇->D2L(q,dq̇),q̇)
end

function D1D2L(q,q̇)
    return ForwardDiff.jacobian(dq->D2L(dq,q̇),q)
end

In [ ]:
function dynamics(x,τ)
    q = x[1:2]
    q̇ = x[3:4]
    
    #Forced Euler-Lagrange Equation
    q̈ = D2D2L(q,q̇)\(D1L(q,q̇) - D1D2L(q,q̇)*q̇ + τ)
    
    ẋ = [q̇; q̈]
end

In [ ]:
function controller(x)
    q = x[1:2]
    q̇ = x[3:4]
    
    kp = 100.0
    kd = 10.0
    
    u = -kp*(q-[pi; 0.0]) - kd*q̇
end

In [ ]:
#Simulate
x0 = randn(4)

function f(x,p,t)
    #u = zeros(2)
    u = controller(x)
    dynamics(x, u)
end

tspan = (0.0,10.0)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5(),abstol=1e-6,reltol=1e-6);

In [ ]:
plot(sol,idxs=(0,1))
plot!(sol,idxs=(0,2))

In [ ]:
function y(q)
    #We're going to apply a force in the x-direction at the end of the 2nd link
    return ℓ*sin(q[1]) + ℓ*sin(q[1]+q[2])
end

function Y(q)
    ForwardDiff.gradient(dq->y(dq),q)'
end

In [ ]:
function dynamics(x,F)
    q = x[1:2]
    q̇ = x[3:4]
    
    #damping constant
    c = 10.0
    
    #Forced Euler-Lagrange Equation
    q̈ = D2D2L(q,q̇)\(D1L(q,q̇) - D1D2L(q,q̇)*q̇ - c*q̇ + Y(q)'*F)
    
    ẋ = [q̇; q̈]
end

In [ ]:
#Simulate
x0 = zeros(4)

function f(x,p,t)
    F = 100.0
    dynamics(x, F)
end

tspan = (0.0,10.0)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5(),abstol=1e-6,reltol=1e-6);

In [ ]:
plot(sol,idxs=(0,1))
plot!(sol,idxs=(0,2))